**Exercise 1: Train a simple trigram language model**
---

-----

In the first exercise, we´ll save the counts directly in a dictionary
 which defaults to the smoothing factor (_note that this is not true smoothing
 as it does not account for the denominator and therefore does not create a
 true probability distribution, but it is enough to get started_)

In [1]:
import nltk
from collections import defaultdict
import numpy as np
import nltk.corpus
from nltk.corpus import brown

# choose a small smoothing factor
smoothing_factor = 0.001
counts = defaultdict(lambda: defaultdict(lambda: smoothing_factor))

We'll also define two helper functions, one to get the log probability of
a single trigram and the second to get the log probability of a full sentence

In [2]:
def logP(u, v, w):
    """
    Compute the log probability of a specific trigram
    """
    return np.log(counts[(u, v)][w]) - np.log(sum(counts[(u, v)].values()))


def sentence_logP(S):
    """
    Adds the special tokens to the beginning and end.
    Then calculates the sum of log probabilities of
    all trigrams in the sentence.
    """
    tokens = ['*', '*'] + S + ['STOP']
    return sum([logP(u, v, w) for u, v, w in nltk.ngrams(tokens, 3)])

We then choose the corpus. We'll use the preprocessed Brown corpus (nltk.corpus.brown), which contains many domains.
To see the domains, you can run brown.categories(). We also split this into train, dev, and test sets, which we will use throughout.

In [17]:
nltk.download('brown')
print(brown.categories())
sentences = brown.sents(categories='news')
dev_idx = int(len(sentences) * .7)
test_idx = int(len(sentences) * .8)
train = sentences[:dev_idx]
dev = sentences[dev_idx:test_idx]
test = sentences[test_idx:]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies', 'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance', 'science_fiction']


Finally, we'll collect the counts in the dictionary we set up before.

In [4]:
for sentence in train:
    # add the special tokens to the sentences
    tokens = ['*', '*'] + sentence + ['STOP']
    for u, v, w in nltk.ngrams(tokens, 3):
        # update the counts
        counts[(u, v)][w] += 1

In [5]:
# Now that we have the model we can use it
print(sentence_logP("what is the best sentence ?".split()))

-42.70140049292593


**Exercise 2: (3-5 minutes)**
---

-----

**Try and find the sentence (len > 10 tokens) with the highest probability**

1. What is the sentence with the highest probability you could find?
2. What is it's log probability?

In [6]:
max = - float("inf")
sent = []
for sentence in sentences:
    p = sentence_logP(sentence)
    if len(sentence) > 10 and p > max:
        max = p
        sent = sentence

print(" ".join(sent), max)

Heating is by individual gas-fired , forced warm air systems . -8.081395957153678


**Exercise 3: Function for trigram model, define perplexity, find the best train domain (15-20 minutes)**
---

-----

First, you'll need to define a function to train the trigram models. It should return the same kind of counts dictionary as in Exercise 1.

In [7]:
def estimate_lm(corpus, smoothing_factor=0.001):
    """This function takes a corpus and returns a trigram model (counts) trained on the corpus """
    counts = defaultdict(lambda: defaultdict(lambda: smoothing_factor))
    for sentence in corpus:
        # add the special tokens to the sentences
        tokens = ['*', '*'] + sentence + ['STOP']
        for u, v, w in nltk.ngrams(tokens, 3):
            # update the counts
            counts[(u, v)][w] += 1
    return counts

Now, you'll need to define a function to measure perplexity, which is defined as the exp(total negative log likelihood / total_number_of_tokens). See https://web.stanford.edu/~jurafsky/slp3/3.pdf for more info.

Luckily, we already have a function to get the log likelihood of a sentence (sentence_logP). So we can iterate over the sentences in a corpus, summing the log probability of each sentence, and keeping track of the total number of tokens. Finally, you can get the NEGATIVE log likelihood and average this, finally using np.exp to exponentiate the previous result.

In [8]:
def perplexity(corpus):
    """
    Perplexity is defined as the exponentiated average negative log-likelihood of a sequence. 
    """
    total_log_likelihood = 0
    total_token_count = 0
    
    for sentence in corpus:
        total_log_likelihood += sentence_logP(sentence)
        total_token_count += len(sentence)
    
    perplexity = np.exp(- total_log_likelihood / total_token_count)
    return perplexity

In [9]:
test_data = [["I'm", 'not', 'giving', 'you', 'a', 'chance', ',', 'Bill', ',', 'but', 'availing', 'myself', 'of', 'your', 'generous', 'offer', 'of', 'assistance', '.'], ['Good', 'luck', 'to', 'you', "''", '.'], ['``', 'All', 'the', 'in-laws', 'have', 'got', 'to', 'have', 'their', 'day', "''", ',', 'Adam', 'said', ',', 'and', 'glared', 'at', 'William', 'and', 'Freddy', 'in', 'turn', '.'], ['Sweat', 'started', 'out', 'on', "William's", 'forehead', ',', 'whether', 'from', 'relief', 'or', 'disquietude', 'he', 'could', 'not', 'tell', '.'], ['Across', 'the', 'table', ',', 'Hamrick', 'saluted', 'him', 'jubilantly', 'with', 'an', 'encircled', 'thumb', 'and', 'forefinger', '.'], ['Nobody', 'else', 'showed', 'pleasure', '.'], ['Spike-haired', ',', 'burly', ',', 'red-faced', ',', 'decked', 'with', 'horn-rimmed', 'glasses', 'and', 'an', 'Ivy', 'League', 'suit', ',', 'Jack', 'Hamrick', 'awaited', 'William', 'at', 'the', "officers'", 'club', '.'], ['``', 'Hello', ',', 'boss', "''", ',', 'he', 'said', ',', 'and', 'grinned', '.'], ['``', 'I', 'suppose', 'I', 'can', 'never', 'expect', 'to', 'call', 'you', "'", 'General', "'", 'after', 'that', 'Washington', 'episode', "''", '.'], ['``', "I'm", 'afraid', 'not', "''", '.']]

Finally, use *estimate_lm()* to train LMs on each domain in brown.categories() and 
find which gives the lowest perplexity on test_data. 

1. Which domain gives the best perplexity?
2. Can you think of a way to use language models to predict domain?

In [10]:
min = float("inf")
dom = ""
for domain in brown.categories():
    train = brown.sents(categories=domain)
    counts = estimate_lm(train)
    perp = perplexity(test_data)
    print(domain, perp)
    if perp < min:
        min = perp
        dom = domain

print("Domain with lowest perplexity:", dom, min)

adventure 17.5115745216489
belles_lettres 23.839993737294026
editorial 14.207547015074889
fiction 19.59697351759496
government 11.068992137052321
hobbies 14.647528814565485
humor 8.230644148155282
learned 18.389202856560622
lore 24.26379000002405
mystery 17.93675812239107
news 18.720360984322056
religion 13.543580787870582
reviews 8.790179135397917
romance 3.2907736822940756
science_fiction 10.460016796365833
Domain with lowest perplexity: romance 3.2907736822940756


**Exercise 4: Generation**
---

-----

For the next exercise, you will need to generate 10 sentences for each domain in the Brown corpus. The first thing we need is code to be able to sample the next word in a trigram. We'll do this by creating a probability distribution over the values in our trigram counts. Remember that each key in the dictionary is a tuple (u, v) and that the values is another dictionary with the count of the continuation w: count. Therefore, we can create a numpy array with the continuation values and divide by the sum of values to get a distribution. Finally, we can use np.random.multinomial to sample from this distribution.

In [13]:
def sample_next_word(u, v):
    keys, values = zip(* counts[(u, v)]. items())
    # convert values to np.array
    values = np.array(values)
    # divide by sum to create prob. distribution
    values /= values.sum()  
    # return the key (continuation token) for the sample with the highest probability
    return keys[np.argmax(np.random.multinomial(1, values))]  

Now we can create a function that will generate text using our trigram model. You will need to start out with the two special tokens we used to train the model, and continue adding to this output, sampling the next word at each timestep. If the word sampled is the end token ('STOP'), then stop the generation and return the sequence as a string.

In [11]:
def generate():
    """
    Sequentially generates text using sample_next_word().
    When the token generated is 'STOP', it returns the generated tokens as a string,
    removing the start and end special tokens.
    """
    result = ['*', '*']
    i = 0
    word = ""
    while word != "STOP":
        word = sample_next_word(result[i], result[i + 1])
        result.append(word)
        i += 1

    clean = result[2:len(result)-1]
    return " ".join(clean)

Finally, use the code above to generate 10 sentences per domain in the Brown corpus.

1. Do you see any correlation between perplexity scores and generated text?

In [14]:
for domain in brown.categories():
    train = brown.sents(categories=domain)
    counts = estimate_lm(train)
    print("DOMAIN:", domain)
    sentences = []
    for _ in range(10):
        sentence = generate()
        sentences.append(sentence.split())
        print(sentence)
    perp = perplexity(sentences)
    print("PERPLEXITY:", perp)
    print()

DOMAIN: adventure
There was no reply so he spent most of the stockade wall just out of a politician as well enjoy it .
A tight wagon meant so much to give them privacy on their way back there when you leave .
The girl asked .
They appeared to disapprove of my background applying at the hall , could only play the last place of any customers walking in at the doctor from the outlaw , kept his face .
The bullet had torn through the rain and across the brushy swells told him .
Morgan laughed .
If he wondered whether the attackers would allow him to pull away unmolested , he doesn't deserve to lie there in this country within 3 days , your time will come along '' , said Tilghman .
He drew a long time that had frightened her .
`` I mean '' --
He supervised the cleanups and handled the shipments of raw gold which each week went out to his lack of fitness for courting .
PERPLEXITY: 3.237558726414217

DOMAIN: belles_lettres
In this way and that , unlike the novelists and poets who had returned 

**Exercise 5: Smoothing**
---

-----

So far, we have been using a kind of stupid smoothing technique, giving up entirely on computing an actual probability distribution. For this section, let's implement a correct version of Laplace smoothing. You'll need to keep track of the vocabulary as well, and don't forget to add the special tokens.

In [19]:
def estimate_lm_smoothed(corpus, alpha=1):
    counts = defaultdict(lambda: defaultdict(lambda: 0))
    vocab = set(['*', 'STOP'])

    for sentence in corpus:
        # add the special tokens to the sentences
        tokens = ['*', '*'] + sentence + ['STOP']
        for token in sentence:
            vocab.add(token)
        for u, v, w in nltk.ngrams(tokens, 3):
            # update the counts
            counts[(u, v)][w] += 1

    return counts, vocab

The main change is not in how we estimate the counts, but in how we calculate log probability for each trigram.
Specifically, we need to add the size_of_the_vocabulary * alpha to the denominator.

In [23]:
def logP_smoothed(u, v, w, V, alpha=1):
    """
    Compute the log probability of a specific trigram
    """
    return np.log(counts[(u, v)][w] + alpha) - np.log(sum(counts[(u, v)].values()) + len(V)*alpha)

def sentence_logP_smoothed(S, V, alpha=1):
    """
    Adds the special tokens to the beginning and end.
    Then calculates the sum of log probabilities of
    all trigrams in the sentence using logP_smoothed.
    """
    tokens = ['*', '*'] + S + ['STOP']
    return sum([logP_smoothed(u, v, w, V, alpha) for u, v, w in nltk.ngrams(tokens, 3)])

def perplexity_smoothed(corpus, V, alpha=1):
    """
    Perplexity is defined as the exponentiated average negative log-likelihood of a sequence. In this case, we approximate perplexity over the full corpus as an average of sentence-wise perplexity scores.
    """
    total_log_likelihood = 0
    total_token_count = 0
    
    for sentence in corpus:
        total_log_likelihood += sentence_logP_smoothed(sentence, V, alpha)
        total_token_count += len(sentence)
    
    perplexity = np.exp(- total_log_likelihood / total_token_count)
    return perplexity

Now train s_counts and vocab and compare perplexity with the original version on the heldout test set.

In [26]:
for domain in brown.categories():
    sentences = brown.sents(categories=domain)
    dev_idx = int(len(sentences) * .7)
    test_idx = int(len(sentences) * .8)
    train = sentences[:dev_idx]
    dev = sentences[dev_idx:test_idx]
    test = sentences[test_idx:]
    train = brown.sents(categories=domain)
    counts = estimate_lm(train)
    perp = perplexity(test)
    print(domain, perp)

adventure 3.62618443256563
belles_lettres 3.9012572602789732
editorial 2.864068521672823
fiction 3.5531091352058013
government 2.928235898815161
hobbies 2.9136126539130376
humor 2.154126063174749
learned 4.061666793956146
lore 3.3842649255871065
mystery 3.642513970189877
news 3.127661915070667
religion 2.876022689107901
reviews 2.288718842401025
romance 3.760835096939757
science_fiction 2.384818394798587


In [24]:
for domain in brown.categories():
    sentences = brown.sents(categories=domain)
    dev_idx = int(len(sentences) * .7)
    test_idx = int(len(sentences) * .8)
    train = sentences[:dev_idx]
    dev = sentences[dev_idx:test_idx]
    test = sentences[test_idx:]
    train = brown.sents(categories=domain)
    counts, vocab = estimate_lm_smoothed(train)
    perp = perplexity_smoothed(test, vocab)
    print(domain, perp)

adventure 4658.570954741525
belles_lettres 9373.49493686855
editorial 5781.57447517779
fiction 4880.377736383577
government 4583.126411354653
hobbies 7110.193799545558
humor 3001.1276205544236
learned 8722.054703487684
lore 7885.851210182688
mystery 4014.3626080576055
news 8364.05170275737
religion 3809.1778548140337
reviews 5372.713225295355
romance 4421.488819829962
science_fiction 2028.2703269245585


**Exercise 6: Find the optimal smoothing factor**
---

-----

Now create a development set using some of the training data and find the optimal smoothing factor and retest on the test set

In [28]:
full_corpus = brown.sents()
train_idx = int(len(full_corpus) * .7)
dev_idx = int(len(full_corpus) * .8)
train = full_corpus[:train_idx]
dev = full_corpus[train_idx:dev_idx]
test = full_corpus[dev_idx:]

best_alpha = 1000
best_perplex = 100000

for alpha in [100, 10, 1, 0.1, 0.01, 0.001, 0.0001]:
    s_counts, vocab = estimate_lm_smoothed(train)
    dev_perplex = perplexity_smoothed(dev, vocab, alpha=alpha)
    if dev_perplex < best_perplex:
        best_alpha = alpha
        best_perplex = dev_perplex
        print("Current best: {0:.2f} with alpha={1}".format(best_perplex,
                                                            best_alpha))

print("Best alpha: {0}".format(best_alpha))
print("Smoothed perplexity: {0:.3f}".format(perplexity_smoothed(test, vocab, alpha=best_alpha)))
print()

Current best: 95829.10 with alpha=100
Current best: 87428.61 with alpha=10
Current best: 72553.08 with alpha=1
Current best: 55528.10 with alpha=0.1
Current best: 40756.69 with alpha=0.01
Current best: 20990.22 with alpha=0.001
Current best: 4709.40 with alpha=0.0001
Best alpha: 0.0001
Smoothed perplexity: 2747.252



**Exercise 7: Interpolation**
---

-----

To be able to interpolate unigram, bigram, and trigram models, we first need to train them. So here you need to make a function that takes 1) a corpus and 2) an n-gram (1,2,3) and 3) a smoothing factor and returns the counts and vocabulary. Notice that for the unigram model, you will have to set up the dictionary in a different way than we have done until now.

In [29]:
def estimate_ngram(corpus, N=3, smoothing_factor=1):
    vocab = set(['*', 'STOP'])
    if N > 1:
        # set up the counts like before
        counts = defaultdict(lambda: defaultdict(lambda: 0))
        for sentence in corpus:
            # add the special tokens to the sentences
            vocab.update(sentence)
            tokens = ['*'] * (N - 1) + sentence + ['STOP']
            for ngram in nltk.ngrams(tokens, N):
                # update the counts
                counts[ngram[:-1]][ngram[-1]] += 1
    else:
        # set them up as necessary for the unigram model
        counts = defaultdict(lambda: 0)
        for sentence in corpus:
            vocab.update(sentence)
            for token in sentence:
                counts[token] += 1
        
    # Finish the code here
    return counts, vocab

You will also need separate functions to get the log probability for each ngram.

In [30]:
def logP_trigram(counts, u, v, w, vocab, alpha=1):
    """
    Compute the log probability of a specific trigram
    """
    return np.log(counts[(u, v)][w] + alpha) - np.log(sum(counts[(u, v)].values()) + len(vocab)*alpha)

def logP_bigram(counts, u, v, vocab, alpha=1):
    """
    Compute the log probability of a specific bigram
    """
    return np.log(counts[u][v] + alpha) - np.log(sum(counts[u].values()) + len(vocab)*alpha)

def logP_unigram(counts, u, vocab, alpha=1):
    """
    Compute the log probability of a specific unigram
    """
    return np.log(counts[u] + alpha) - np.log(sum(counts.values()) + len(vocab)*alpha)

In this case, the main change is in calculating the log probability of the sentence. 

In [34]:
def sentence_interpolated_logP(S, vocab, uni_counts, bi_counts, tri_counts, lambdas=[0.5, 0.3, 0.2], alpha=1):
    tokens = ['*', '*'] + S + ['STOP']
    prob = 0
    # Calculate the log probabilities for each ngram and then multiply them by the lambdas and sum them.
    for u, v, w in nltk.ngrams(tokens, 3):
        tri_prob = logP_trigram(tri_counts, u, v, w, vocab, alpha)
        bi_prob = logP_bigram(bi_counts, u, v, vocab, alpha)
        uni_prob = logP_unigram(uni_counts, u, vocab, alpha)
        prob += lambdas[0] * tri_prob + lambdas[1] * bi_prob + lambdas[2] * uni_prob

    return prob

def interpolated_perplexity(corpus, vocab, uni_counts, bi_counts, tri_counts, lambdas=[0.5, 0.3, 0.2], alpha=1):
    """
    Perplexity is defined as the exponentiated average negative log-likelihood of a sequence. 
    In this case, we approximate perplexity over the full corpus as an average of sentence-wise perplexity scores.
    """
    total_log_likelihood = 0
    total_token_count = 0
    
    for sentence in corpus:
        total_log_likelihood += sentence_interpolated_logP(sentence, vocab, uni_counts, bi_counts, tri_counts, lambdas, alpha)
        total_token_count += len(sentence)
    
    perplexity = np.exp(- total_log_likelihood / total_token_count)
    return perplexity

Finally, train unigram, bigram, and trigram models and compute the perplexity of the interpolated model on the test set.

In [36]:
for domain in brown.categories():
    sentences = brown.sents(categories=domain)
    dev_idx = int(len(sentences) * .7)
    test_idx = int(len(sentences) * .8)
    train = sentences[:dev_idx]
    dev = sentences[dev_idx:test_idx]
    test = sentences[test_idx:]
    train = brown.sents(categories=domain)
    uni_counts, vocab = estimate_ngram(train, N=1)
    bi_counts, vocab = estimate_ngram(train, N=2)
    tri_counts, vocab = estimate_ngram(train, N=3)
    perp = interpolated_perplexity(test, vocab, uni_counts, bi_counts, tri_counts, alpha=best_alpha)
    print(domain, perp)

adventure 250.96455308415733
belles_lettres 304.05248614516256
editorial 208.6668680699024
fiction 227.41646083281498
government 185.91414516263308
hobbies 245.40898649261095
humor 122.90472507564117
learned 331.29422014674515
lore 258.66422440150126
mystery 233.26343190817988
news 271.2335662862673
religion 188.12725095094245
reviews 183.30263047121545
romance 244.66821364231544
science_fiction 122.09443100947084


**Exercise 8: Build a simple spelling corrector**
---

-----

In this section, we will build a simple spelling corrector with two components: 1) a dictionary of common spelling errors which will allow us to create possible hypothesis sentences and 2) a language model to filter the most likely sentence.

In [40]:
common_errors = {"ei": "ie",  # acheive: achieve
                 "ie": "ei",  # recieve: receive
                 "ant": "ent",  # apparant: apparent
                 "m": "mm",  # accomodate: accommodate
                 "s": "ss",  # profesional: professional
                 "teh": "the",
                 "too": "to",
                 "their": "there",
                 "there": "they're"
                 }

In [59]:
test_set = ["I do not know who acheived it".split(),
            "I do not know who recieved it".split(),
            "That is not apparant".split(),
            "We can only accomodate one person".split(),
            "That is not profesional".split(),
            "we saw teh man running".split(),
            "We tried too help them".split(),
            "their is a good weather".split(),
            "there my friends".split(),
            ]

For the spell checker,

In [56]:
def spell_check(sent, common_errors):
    sents = []
    probs = []
    sents.append(sent)
    probs.append(sentence_logP(sent))
    
    # create new hypothesis sentences by recursively applying all possible spelling mistakes to 
    # each token in the sentence. If the new sentence is not the same as the original, append
    # it to sents and compute its probability and append it to probs.
    for i, token in enumerate(sent):
        for error, correct in common_errors.items():
            if token.find(error) != -1:
                new_token = token.replace(error, correct, 1)
                new_sent = sent.copy()
                new_sent[i] = new_token
                sents.append(new_sent)
                probs.append(sentence_logP(new_sent))
        
    # Finally take the argmax of the probabilities and return that sentence
    # max_i = np.argmin(probs)
    # return sents[max_i], probs[max_i]
    min_i = np.argmin(probs)
    return sents[min_i], probs[min_i]

It would be a good idea to retrain your langauge model on all of the Brown sentences (brown.sents()) in order to improve it's recall.

1. After retraining, do you notice any differences?

In [60]:
train = brown.sents()
counts = estimate_lm(train)

In [61]:
for sent in test_set:
    sent_check, prob = spell_check(sent, common_errors)
    print("Original:", " ".join(sent), sentence_logP(sent))
    print("Correct:", " ".join(sent_check), prob)

Original: I do not know who acheived it -23.962760536913688
Correct: I do not know who achieved it -31.564662496788856
Original: I do not know who recieved it -23.963159977701903
Correct: I do not know who received it -32.48155244969362
Original: That is not apparant -23.60046156453007
Correct: That is not apparent -23.60046156453007
Original: We can only accomodate one person -38.29513646861539
Correct: We can only accommodate one person -37.60198928805545
Original: That is not profesional -23.600465386977394
Correct: That is not professional -30.510218668622205
Original: we saw teh man running -26.853802458286175
Correct: we saw the man running -35.17095969143966
Original: We tried too help them -34.57383974386029
Correct: We tried to help them -42.40575643176704
Original: their is a good weather -41.53606320227949
Correct: their is a good weather -41.53597994515282
Original: there my friends -27.664014374023065
Correct: they're my friends -26.566130361233142
